In [1]:
import pandas as pd, numpy as np

In [2]:
seed = 1
np.random.seed(seed)

## Load data

In [3]:
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")
train_df.shape, test_df.shape

((4209, 378), (4209, 377))

In [5]:
train_df.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [10]:
test_df.head()

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0


## Preprocess data

In [4]:
from sklearn.preprocessing import LabelEncoder

In [11]:
# train_input = train_df.drop("ID") First, experiment with keeping id
train_input = train_df.copy()
test_input = test_df.copy()

In [12]:
num_train =len( train_input)
num_train

4209

In [13]:
full_input = pd.concat([train_input, test_input])
full_input.shape

(8418, 378)

In [17]:
full_input = pd.get_dummies(full_input)
full_input.shape

(8418, 581)

In [22]:
train_input = full_input[:num_train]
test_input = full_input[num_train:]
test_input.drop('y', axis=1,inplace=True)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## model

In [31]:
from keras.models import Sequential, Model
from keras.layers import Dense, Multiply, Input, Dropout
from keras.layers.normalization import BatchNormalization
from keras import backend as K

In [26]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [27]:
input_dim = test_input.shape[1]

In [47]:
all_inputs = Input(shape=(input_dim,))
attention_probs = Dense(input_dim, activation='softmax', name='attention_probs')(all_inputs)
attended_inputs = Multiply()([all_inputs, attention_probs])

x = Dense(256, activation='relu')(attended_inputs)
x = Dropout(0.4)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)
output = Dense(1, activation='relu')(x)

model = Model(inputs=all_inputs, outputs=output)
model.compile(loss='mse', optimizer='adam', metrics=[r2_keras])

In [48]:
from keras_tqdm import TQDMNotebookCallback
from keras.callbacks import EarlyStopping

In [49]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10),
    TQDMNotebookCallback()
]

In [50]:
val_split = 0.2

In [51]:
model.fit(train_input.drop('y', axis=1).values, train_input['y'].values, epochs = 100, callbacks=callbacks, verbose=0, validation_split=val_split)